In [1]:
import operator as op
import random
import queue
import copy
import numpy as np
from scipy.special import softmax
from Load_data import load_data
#from Load_MNIST import load_MNIST
import math
import pickle
import os

In [2]:
class Function:
    def __init__(self, f, arity, name):
        self.f = f
        self.arity = arity
        self.name = name
    def __call__(self, *args, **kwargs):
        return self.f(*args, **kwargs)

def protected_div(a, b):
    if abs(b) < 1e-6:
        return a
    return a / b

def avg(a, b):
    return (a + b)/2

def protected_sqrt(a):
    return math.sqrt(abs(a))

def square(a):
    return a*a

In [3]:
def init_function_matrix(n_row, n_col, function_set):
    '''
    To represent an individual in CGP population.
    input:
        n_row: scalar, number of rows
        n_col: scalar, number of columns
        function_set: set, a group of functions to be chosen from
    output:
        function_matrix: a matrix which has shape (n_row, n_col)
    '''
    function_matrix = []
    for i in range(n_row):
        temp = []
        for j in range(n_col):
            temp.append(random.choice(function_set))
        function_matrix.append(temp)
    return function_matrix

In [4]:
def read_function_matrix(function_matrix):
    f_m = []
    for i in range(len(function_matrix)):
        temp = []
        for j in range(len(function_matrix[i])):
            temp.append(function_matrix[i][j].name)
        f_m.append(temp)
    return f_m

function_matrix = init_function_matrix(n_row = 3, n_col = 10, function_set = function_set)

f_m = read_function_matrix(function_matrix)

f_m

In [5]:
def init_connect_matrix(n_input, n_output, level_back, function_matrix):
    '''
    To record connectivity in an individual. The first row 
    input:
        n_input: scalar, number of inpput
        n_output: scalar, number of output
        level_back: scalar, indicate how many layers two nodes can get connected
        function_matrix: a matrix which records functions
    output:
        connect_matrix: a matrix which records connections between input and node, node and node, and, node and output
    '''
    connect_matrix = []
    for i in range(len(function_matrix)):
        temp1 = []
        for j in range(len(function_matrix[i])):
            temp2 = []
            for k in range(function_matrix[i][j].arity):
                back_row = random.randint(max(-1, i - level_back), i - 1)                            
                if back_row == -1:
                    coordinate = (-1, random.randint(0, n_input - 1))
                    temp2.append(coordinate)
                else:
                    coordinate = (back_row, random.randint(0, len(function_matrix[back_row]) - 1))
                    temp2.append(coordinate)
            temp1.append(temp2)
        connect_matrix.append(temp1)
    temp4 = []
    for m in range(n_output):
        temp3 = []
        back_row = random.randint(max(-1, len(function_matrix) - level_back), len(function_matrix) - 1)
        coordinate = (back_row, random.randint(0, len(function_matrix[back_row]) - 1))
        temp3.append(coordinate)
        temp4.append(temp3)
    connect_matrix.append(temp4)
    return connect_matrix

connect_matrix = init_connect_matrix(n_input = 50, n_output = 10, level_back = 2, function_matrix = function_matrix)

connect_matrix

In [6]:
def init_active_matrix(n_input, connect_matrix):
    '''
    To record which node is activated.
    input:
        connect_matrix: a matrix which records connections between input and node, node and node, and, node and output
    output:
        active_matrix: a matrix which has the same shape (n_row, n_col) as function matrix; '1' indicates active and '0' indicate inactive
    '''
    active_matrix = []
    temp1 = []
    for i in range(n_input):
        temp1.append(0)
    active_matrix.append(temp1)
    for i in range(len(connect_matrix) - 1):
        temp2 = []
        for j in range(len(connect_matrix[i])):
            temp2.append(0)
        active_matrix.append(temp2)
    #--------------------------------------------------
    
    route = queue.Queue()
    for i in connect_matrix[-1]:
        for j in i:
            route.put(j)
    while route.empty() == False:
        (r1, c1) = route.get()
        active_matrix[r1 + 1][c1] = active_matrix[r1 + 1][c1] + 1
        if r1 == -1:
            continue
        else:
            for i in connect_matrix[r1][c1]:
                route.put(i)
        
    return active_matrix

active_matrix = init_active_matrix(n_input = 50, connect_matrix = connect_matrix)

active_matrix

In [7]:
def init_value_matrix(x, active_matrix, connect_matrix, function_matrix):
    '''
    input:
        x: list/array, the input to CGP
        active_matrix: a matrix which has the same shape (n_row, n_col) as function matrix; '1' indicates active and '0' indicate inactive
        connect_matrix: a matrix which records connections between input and node, node and node, and, node and output
        function_matrix: a matrix which has shape (n_row, n_col)
    output:
        value_matrix: a matrix which has the same shape as connect_matrix
    '''
    value_matrix = []
    for i in range(len(connect_matrix)):
        temp1 = []
        for j in range(len(connect_matrix[i])):
            if i == len(function_matrix):
                (r, c) = connect_matrix[i][j][0]
                temp1.append(value_matrix[r][c])
            elif active_matrix[i + 1][j] == 0:
                temp1.append(None)
            else:
                inputs = []
                for (r, c) in connect_matrix[i][j]:
                    if r == -1:
                        inputs.append(x[c])
                    else:
                        inputs.append(value_matrix[r][c])
                temp1.append(function_matrix[i][j](*inputs))
        value_matrix.append(temp1)
    return value_matrix

x = [i for i in range(50)]

value_matrix = init_value_matrix(x = x, active_matrix = active_matrix, connect_matrix = connect_matrix, function_matrix = function_matrix)

value_matrix

In [8]:
def init_confuse_matrix():
    confuse_matrix = []
    for i in range(10):
        temp1 = []
        for j in range(10):
            temp1.append(0)
        confuse_matrix.append(temp1)
    return confuse_matrix

In [9]:
class Individual():
    '''
    用于产生CGP中的个体
    '''
    def __init__(self):
        self.fitness = 0
        self.generation = 0
        self.function_matrix = init_function_matrix(n_row, n_col, function_set)
        self.connect_matrix = init_connect_matrix(n_input, n_output, level_back, self.function_matrix)
        self.active_matrix = init_active_matrix(n_input, self.connect_matrix)
        #self.value_matrix = init_value_matrix(x, self.active_matrix, self.connect_matrix, self.function_matrix)

In [10]:
def evaluate(indiv, X_train, Y_train):
    '''
    input:
        indiv: Individual
        X_train: array/list, all samples which are injected into CGP
        Y_train: array/list, labels
    output:
        fitness: scalar, how many labels are successfully predicted
        confuse_matrix: a matrix indicate which labels are wrongly predicted
    '''
    indiv.fitness = 0
    indiv.confuse_matrix = init_confuse_matrix()
    for x, y in zip(X_train, Y_train):
        indiv.value_matrix = init_value_matrix(x, indiv.active_matrix, indiv.connect_matrix, indiv.function_matrix)
        possibility = softmax(indiv.value_matrix[-1])
        #y_hat是预测值
        y_hat = np.argmax(possibility)
        if int(y) == y_hat:
            indiv.fitness = indiv.fitness + 1
            indiv.confuse_matrix[int(y)][y_hat] = indiv.confuse_matrix[int(y)][y_hat] + 1
        else:
            indiv.confuse_matrix[int(y)][y_hat] = indiv.confuse_matrix[int(y)][y_hat] + 1
    return indiv

pa = Individual()

fitness, confuse_matrix = evaluate(pa, X, Y)

confuse_matrix

In [11]:
def mutate(parent, probability):
    '''
    input:
        parent: Individual, has function_matrix, connect_matrix and active_matrix
    output:
        child: Individual, has function_matrix, connect_matrix and active_matrix
    '''
    child = copy.deepcopy(parent)
    for i in range(len(child.connect_matrix)):
        for j in range(len(child.connect_matrix[i])):
            coin = random.random()
            if coin < probability:
                if i == len(child.function_matrix):
                    back_row = random.randint(max(-1, i - level_back), i - 1)
                    coordinate = (back_row, random.randint(0, len(child.function_matrix[back_row]) - 1))
                    child.connect_matrix[i][j] = [coordinate]
                else:
                    child.function_matrix[i][j] = random.choice(function_set)
                    coordinates = []
                    for k in range(child.function_matrix[i][j].arity):
                        back_row = random.randint(max(-1, i - level_back), i - 1)                            
                        if back_row == -1:
                            coordinates.append((-1, random.randint(0, n_input - 1)))
                        else:
                            coordinates.append((back_row, random.randint(0, len(child.function_matrix[back_row]) - 1)))
                    child.connect_matrix[i][j] = coordinates
            else:
                continue
    child.active_matrix = init_active_matrix(n_input, child.connect_matrix)
    return child

ch = mutate(parent = pa, probability = 0.4)

In [12]:
def population(n_indiv):
    pop = []
    #temp_acc = []
    for _ in range(n_indiv):
        indiv = Individual()
        indiv = evaluate(indiv, X_train, Y_train)
        #temp_acc.append(indiv.fitness/len(X))
        pop.append(indiv)
    pop = sorted(pop, key=lambda individual: individual.fitness)
    #temp_acc = sorted(temp_acc)
    #accuracy.append(temp_acc)
    with open(pop_history_file + '0', 'wb') as f:
        pickle.dump(pop, f)
    return pop

In [13]:
def evolve(pop, start_gen, gen_per):
    '''
    To evolve population by generations.
    input:
        pop: list, population
        start_gen: scalar, the index of the first generation after the original
        max_gen: scalar, the index of the final generation 
    output:
        pop: the final generation
    '''
    '''
    for gen in range(start_gen, start_gen + gen_per):
        pop = pop[best_num: ]
        temp_acc = accuracy[-1][best_num: ]
        parents = pop[-best_num: ]
        for parent in parents:
            child = mutate(parent, probability)
            child = evaluate(child, X, Y)
            child.generation = gen
            pop.append(child)
            temp_acc.append(child.fitness/len(X))
        pop = sorted(pop, key=lambda individual: individual.fitness)
        temp_acc = sorted(temp_acc)
        accuracy.append(temp_acc)
    '''
    
    #-------------------------------------------------------------------------------------
    for gen in range(start_gen, start_gen + gen_per):
        new_pop = []
        #temp_acc = []
        p1 = softmax([indiv.fitness/len(X_train) for indiv in pop])
        while len(new_pop) < len(pop):
            round_1 = np.random.choice(pop, round_1_size, replace=False, p=p1)
            p2 = softmax([indiv.fitness/len(X_train) for indiv in round_1])
            parent = np.random.choice(round_1, round_2_size, replace=False, p=p2)[0]
            child = mutate(parent, probability)
            child = evaluate(child, X_train, Y_train)
            child.generation = gen
            new_pop.append(child)
            new_pop.append(parent)
            #temp_acc.append(child.fitness/len(X))
            #temp_acc.append(parent.fitness/len(X))
        pop = sorted(new_pop, key=lambda individual: individual.fitness)
        with open(pop_history_file + str(gen), 'wb') as f:
            pickle.dump(pop, f)
        #temp_acc = sorted(temp_acc)
        #accuracy.append(temp_acc)
    return pop

In [17]:
n_row = 4
n_col = 15
function_set = [Function(op.add, 2, 'add'),
                Function(op.sub, 2, 'sub'),
                Function(op.mul, 2, 'mul'),
                Function(op.neg, 1, 'neg'),
                Function(protected_div, 2, 'div'),
                Function(avg, 2, 'avg'),
                Function(op.abs, 1, 'abs'),
                Function(max, 2, 'max'),
                Function(min, 2, 'min'),
                Function(math.sin, 1, 'sin'),
                Function(math.cos, 1, 'cos'),
                Function(protected_sqrt, 1, 'sqrt'),
                Function(square, 1, 'square'),
               ]
X_train, Y_train, X_test, Y_test = load_data()
'''X_train, Y_train, X_test, Y_test = load_MNIST()
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)'''
n_input = len(X_train[0])
n_output = 10
level_back = 2
probability = 0.05
n_indiv = 20
round_1_size = 5
round_2_size = 1
pop_history_file = './pop_history/2021_02_16-8/'
start_gen = len(os.listdir(pop_history_file))
gen_per = 15000
proceed = False

In [18]:
if proceed:
    print('continue from ' + str(start_gen - 1))
    with open(pop_history_file + str(start_gen - 1), 'rb') as f:
        pop = pickle.load(f)
else:
    print('create new population')
    pop = population(n_indiv)
    start_gen = start_gen + 1
    print('done!')

create new population
done!


In [19]:
pop = evolve(pop, start_gen, gen_per)
start_gen = gen_per + start_gen